In [1]:
import pandas as pd
import numpy as np
import pyodbc
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
from scipy.interpolate import interp1d

import requests
import folium

In [2]:
# Configuración de los parámetros de conexión
server = 'localhost\\SQLEXPRESS'  # O el punto '.' que usaste en SSMS
database = 'EM_BOG'

# Cadena de conexión usando Autenticación de Windows (Trusted_Connection)
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;'
)


In [3]:
query = "select * from dbo.estaciones_full;"
conn = pyodbc.connect(conn_str)
raw_data = pd.read_sql(query, conn)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_10216\341712462.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_data = pd.read_sql(query, conn)


In [4]:
query = "select * from dbo.coordenadas_estaciones;"
conn = pyodbc.connect(conn_str)
coor = pd.read_sql(query, conn)
coor.drop(["tx_id"], axis=1, inplace=True)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_10216\2772575436.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coor = pd.read_sql(query, conn)


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18332 entries, 0 to 18331
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   tx_id              18332 non-null  int64         
 1   codigo_estacion    18332 non-null  object        
 2   fecha_observacion  18332 non-null  datetime64[ns]
 3   sensor_pre         18332 non-null  object        
 4   sensor_vel         18332 non-null  object        
 5   sensor_dir         18332 non-null  object        
 6   sensor_tem         18332 non-null  object        
 7   presion            18332 non-null  float64       
 8   velocidad          18332 non-null  float64       
 9   direccion          18332 non-null  float64       
 10  temperatura        18332 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 1.5+ MB


# explo

## coordenadas

In [6]:
coor.apply(lambda df: f"{df["codigo_estacion"]}: ( {df["latitud"]} , {df["longitud"]} )", axis=1)

0    0021205012: ( 4.637 , -74.089 )
1    0021205509: ( 4.397 , -74.628 )
2    0021205710: ( 4.668 , -74.101 )
3    0021205791: ( 4.706 , -74.151 )
4    0021206560: ( 4.625 , -74.156 )
5    0021206600: ( 4.787 , -74.102 )
6    0021206920: ( 4.356 , -74.159 )
7       0035025502: ( 4.56 , -74.0 )
8    2120500204: ( 4.622 , -74.103 )
dtype: object

## registro fechas

In [49]:
# raw_data.groupby(["codigo_estacion", "fecha_observacion"])[["presion", "velocidad", "direccion", "temperatura"]].mean()

In [50]:
df = raw_data.copy()
df.drop("tx_id", axis=1, inplace=True)

In [53]:
print(df.shape)
df = df.groupby(["codigo_estacion", "fecha_observacion"])[["presion", "velocidad", "direccion", "temperatura"]].mean().reset_index()
print(df.shape)

(18332, 10)
(9533, 6)


In [54]:
df

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura
0,0021205012,2025-12-01 00:00:00,751.7,1.2,69.6,14.2
1,0021205012,2025-12-01 01:00:00,752.4,0.7,69.2,13.5
2,0021205012,2025-12-01 02:00:00,752.9,1.0,29.4,12.6
3,0021205012,2025-12-01 03:00:00,753.2,0.9,52.4,11.8
4,0021205012,2025-12-01 04:00:00,753.1,0.4,19.9,12.1
...,...,...,...,...,...,...
9528,2120500204,2025-12-31 19:00:00,752.4,3.8,149.0,18.5
9529,2120500204,2025-12-31 20:00:00,751.6,2.4,165.0,21.0
9530,2120500204,2025-12-31 21:00:00,751.6,2.8,103.0,19.2
9531,2120500204,2025-12-31 22:00:00,751.6,2.9,129.0,19.6


In [ ]:
# print(df.shape)
# print("-"*50)
# cols = ["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"]
# for col in cols:
#     print(df[col].value_counts())
#     col_0 = df[col].value_counts().index[0]
#     df = df[df[col] == col_0].copy()
#     print("-"*50)
# print(df.shape)

(18332, 10)
--------------------------------------------------
sensor_pre
0258    10268
0255     8064
Name: count, dtype: int64
--------------------------------------------------
sensor_vel
0111    5134
0103    5134
Name: count, dtype: int64
--------------------------------------------------
sensor_dir
0104    5134
Name: count, dtype: int64
--------------------------------------------------
sensor_tem
0071    4401
0068     733
Name: count, dtype: int64
--------------------------------------------------
(4401, 10)


In [56]:
estaciones = df["codigo_estacion"].unique()

In [57]:
# df = df.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)
df = df.sort_values(["fecha_observacion", "codigo_estacion"]).reset_index(drop=True)

In [58]:
df["fecha_observacion"].diff().unique()

<TimedeltaArray>
[              NaT, '0 days 00:00:00', '0 days 00:10:00', '0 days 00:20:00',
 '0 days 00:50:00', '0 days 01:00:00']
Length: 6, dtype: timedelta64[ns]

In [59]:
# fecha_minima = pd.to_datetime("2025-12-15")
# df = df[df["fecha_observacion"] >= fecha_minima]

In [60]:
estaciones_ok = list()
n = 1
for estacion in estaciones:
    min_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].min()
    max_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].max()
    diff_temp = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].diff().unique()
    rows_temp = df.loc[df["codigo_estacion"] == estacion].shape[0]
    # if len(diff_temp) == 2: 
    # if rows_temp == 744 and not estacion in ["0024035250", "0024035360"]:
    # if rows_temp == 744:
    # if rows_temp == 408:
    # if min_date <= pd.to_datetime("2025-12-14") and len(diff_temp) == 2:
    # if rows_temp == 408 and not estacion in ["0024035250"]:
    if True:
        print(n , estacion)
        estaciones_ok.append(estacion)
        for diff in diff_temp: print(">", diff)
        print("cantidad de registros: ", rows_temp)
        print("fecha min: ", min_date)
        print("fecha max: ", max_date)
        print("-"*50)
        n += 1

1 0021205012
> NaT
> 0 days 01:00:00
> 0 days 02:00:00
cantidad de registros:  743
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
2 0021205509
> NaT
> 0 days 01:00:00
> 0 days 02:00:00
> 0 days 11:00:00
> 0 days 07:00:00
> 0 days 00:10:00
cantidad de registros:  706
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
3 0021205710
> NaT
> 0 days 01:00:00
> 0 days 06:00:00
> 0 days 05:00:00
cantidad de registros:  735
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
4 0021205791
> NaT
> 0 days 00:10:00
> 0 days 00:20:00
> 0 days 00:50:00
> 0 days 01:00:00
cantidad de registros:  4412
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:50:00
--------------------------------------------------
5 0021206560
> NaT
> 0 days 01:00:00
> 0 days 02:00:00
> 0 days 03:00:00
cantidad de registros:  72

# Elevación

In [61]:
def obtener_alturas_batch(coordenadas):
    """
    Consulta la elevación de múltiples puntos usando la API de Open-Elevation.
    coordenadas: Lista de diccionarios [{'latitude': lat, 'longitude': lon}, ...]
    """
    url = "https://api.open-elevation.com/api/v1/lookup"
    payload = {"locations": coordenadas}
    
    try:
        response = requests.post(url, json=payload, timeout=20)
        if response.status_code == 200:
            return response.json()['results']
    except Exception as e:
        print(f"Error al consultar la API: {e}")
    return None

In [62]:
puntos = list()
for idx in range(len(coor)):
    puntos.append({"latitude": coor.loc[idx, "latitud"], "longitude": coor.loc[idx, "longitud"]})
    ...

In [63]:
resultados = obtener_alturas_batch(puntos)

In [64]:
resultados_dics = {"latitud":list(), "longitud":list(), "altura":list()}
resultados

for resultado in resultados:
    resultados_dics["latitud"].append(resultado["latitude"])
    resultados_dics["longitud"].append(resultado["longitude"])
    resultados_dics["altura"].append(resultado["elevation"])

In [65]:
coor_alt = pd.DataFrame(resultados_dics)

In [67]:
print(f"rows: {df.shape[0]:,}")
wsdata = df.loc[df["codigo_estacion"].isin(estaciones_ok)].copy()
# wsdata.drop(["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"], axis=1, inplace=True)
print(f"rows: {wsdata.shape[0]:,}")
print(f"{100 * wsdata.shape[0] / df.shape[0]:,.2f} %")

rows: 9,533
rows: 9,533
100.00 %


# WSDATA

In [68]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura
0,0021205012,2025-12-01,751.7000,1.200000,69.6,14.200000
1,0021205509,2025-12-01,705.4773,5.070004,110.0,7.367853
2,0021205710,2025-12-01,750.5000,0.700000,95.0,14.300000
3,0021205791,2025-12-01,751.0000,0.900000,215.9,14.500000
4,0021206560,2025-12-01,750.2000,1.000000,317.3,15.100000


In [69]:
wsdata = wsdata.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [70]:
wsdata["longitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["longitud"])))
wsdata["latitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["latitud"])))

In [71]:
wsdata = pd.merge(
    left=wsdata,
    right=coor_alt,
    how="left",
    on=["latitud", "longitud"]
)

In [72]:
wsdata.drop_duplicates(inplace=True)
wsdata.reset_index(drop=True, inplace=True)

In [73]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura,longitud,latitud,altura
0,0021205012,2025-12-01 00:00:00,751.7,1.2,69.6,14.2,-74.089,4.637,2554.0
1,0021205012,2025-12-01 01:00:00,752.4,0.7,69.2,13.5,-74.089,4.637,2554.0
2,0021205012,2025-12-01 02:00:00,752.9,1.0,29.4,12.6,-74.089,4.637,2554.0
3,0021205012,2025-12-01 03:00:00,753.2,0.9,52.4,11.8,-74.089,4.637,2554.0
4,0021205012,2025-12-01 04:00:00,753.1,0.4,19.9,12.1,-74.089,4.637,2554.0


In [74]:
wsdata["fecha_observacion"].sort_values().diff().value_counts()

fecha_observacion
0 days 00:00:00    5121
0 days 00:10:00    4397
0 days 01:00:00       8
0 days 00:20:00       4
0 days 00:50:00       2
Name: count, dtype: int64

In [75]:
fecha_inicio = wsdata["fecha_observacion"].min()
fecha_fin = wsdata["fecha_observacion"].max()
print(f"{fecha_inicio=} , {fecha_fin=}")
rango_tiempo = pd.date_range(fecha_inicio, fecha_fin, freq="10min").to_list()
estaciones_registradas = wsdata["codigo_estacion"].unique()
len(estaciones_registradas)


fecha_inicio=Timestamp('2025-12-01 00:00:00') , fecha_fin=Timestamp('2025-12-31 23:50:00')


8

# Completar fechas con NaN

In [76]:
estaciones_registradas = wsdata["codigo_estacion"].unique()
estaciones = list()
latitudes = list()
longitudes = list()
alturas = list()

for estacion in estaciones_registradas:
    estaciones += [estacion]*len(rango_tiempo)
    latitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "latitud"].unique()[0]]*len(rango_tiempo)
    longitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "longitud"].unique()[0]]*len(rango_tiempo)
    alturas += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "altura"].unique()[0]]*len(rango_tiempo)
    
    ...

fechas = rango_tiempo * len(estaciones_registradas)

In [77]:
base_estacion_fechas = pd.DataFrame({"codigo_estacion":estaciones, "fecha_observacion":fechas, "latitud":latitudes, "longitud":longitudes, "altura":alturas})
wsdata_full = pd.merge(left=base_estacion_fechas, right=wsdata, on=["codigo_estacion", "fecha_observacion", "latitud", "longitud", "altura"], how="left")

In [78]:
wsdata_full.head()

,codigo_estacion,fecha_observacion,latitud,longitud,altura,presion,velocidad,direccion,temperatura
0,0021205012,2025-12-01 00:00:00,4.637,-74.089,2554.0,751.7,1.2,69.6,14.2
1,0021205012,2025-12-01 00:10:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN
2,0021205012,2025-12-01 00:20:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN
3,0021205012,2025-12-01 00:30:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN
4,0021205012,2025-12-01 00:40:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN


In [79]:
wsdata_full = wsdata_full.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [80]:
wsdata_full["vel_u"] = wsdata_full.apply(lambda df: df["velocidad"] * np.sin(np.pi*(df["direccion"]/180)), axis=1)
wsdata_full["vel_v"] = wsdata_full.apply(lambda df: df["velocidad"] * np.cos(np.pi*(df["direccion"]/180)), axis=1)

In [81]:
wsdata_full["segundos"] = (wsdata_full["fecha_observacion"] - fecha_inicio) / np.timedelta64(1, 's')

In [82]:
wsdata_full = wsdata_full[[
    'codigo_estacion',
    'latitud',
    'longitud',
    'altura',
    'segundos',
    'presion',
    'vel_u',
    'vel_v',
    'temperatura',
]].copy()

In [83]:
wsdata_full.sample(5)

,codigo_estacion,latitud,longitud,altura,segundos,presion,vel_u,vel_v,temperatura
33416,2120500204,4.622,-74.103,2558.0,1300800.0,NaN,NaN,NaN,NaN
7552,0021205509,4.397,-74.628,380.0,1852800.0,NaN,NaN,NaN,NaN
20112,0021206560,4.625,-74.156,2550.0,1353600.0,751.9,1.588148,0.606455,19.1
22487,0021206600,4.787,-74.102,2553.0,100200.0,NaN,NaN,NaN,NaN
32913,2120500204,4.622,-74.103,2558.0,999000.0,NaN,NaN,NaN,NaN


In [84]:
path_save_wsdata = Path().cwd().parent / "data" / "raw" / "em_bgta_251201_251231.parquet"
path_save_wsdata_full = Path().cwd().parent / "data" / "raw" / "em_bgta_251201_251231_full.parquet"
wsdata.to_parquet(path_save_wsdata, engine="fastparquet", index=False)
wsdata_full.to_parquet(path_save_wsdata_full, engine="fastparquet", index=False)